In [1]:
import pickle
import numpy as np
import pandas as pd

from gensim.models import Word2Vec
import faiss

In [2]:
# data path
DATA_PATH = 'res/data/new_feat_df.csv'

cleaned_arr_title = pd.read_csv(DATA_PATH, usecols=['cleaned_title']).fillna('').values
arr_weight = pd.read_csv(DATA_PATH, usecols=['weight']).values

## 1. Make wordvector summation (SIF or WVS)

In [3]:
# from sif_generator import SIFGenerator

# sif_generator = SIFGenerator()
# total_wvs = sif_generator.do(cleaned_arr_title, n_worker = 3, svd=True)

In [4]:
from wvs_generator import WVSGenerator

# ex) ['아버지 가방에 들어가신다', '이 문장은 예시 입니다.']
flatten_cleaned_arr_title = cleaned_arr_title.flatten()

wvs_generator = WVSGenerator()
total_wvs = wvs_generator.do(flatten_cleaned_arr_title, n_worker=3)

=> start training w2v model....
=> end training w2v model....
=> start training tfidf model....
=> end training tfidf model....
3 process running...


100%|██████████| 101999/101999 [03:15<00:00, 522.21it/s]


=> make_wvs process is done....


## 2. Save Models & Vectors

In [5]:
WVS_PATH = 'res/data/가공식품_wvs.npz'
W2V_PATH = 'res/model/가공식품_word2vec.model'
TFIDF_PATH = 'res/model/가공식품_tfidf_model.pkl'

# save total wvs vectors
wvs_generator.save_total_wvs(WVS_PATH)

# save w2v_model
wvs_generator.save_w2v_model(W2V_PATH)

# save tfidf_model
wvs_generator.save_tfidf_model(TFIDF_PATH)

=> save total_wvs to res/data/가공식품_wvs.npz
=> save w2v_model to res/model/가공식품_word2vec.model
=> save tfidf_model to res/model/가공식품_tfidf_model.pkl


## 3. Make concat/img vector
-

## 4. Make faiss index

In [6]:
from index_handle import IndexHandler

# vector path
arr_title_vector_path = 'res/data/가공식품_wvs.npz'
arr_img_vector_path = 'res/data/가공식품_IMGF.npz'
arr_concat_vector_path = 'res/data/CONCAT_가공식품.npz'


# load vector (title / img / concat)
arr_title_vector = np.load(arr_title_vector_path)['arr_0']
arr_img_vector = np.load(arr_img_vector_path)['arr_0']
arr_concat_vector = np.load(arr_concat_vector_path)['arr_0']

# make concat vector 
# contig_query_vector = np.ascontiguousarray(query_vector, dtype=np.float32)
# contig_img_vector = np.ascontiguousarray(img_vector, dtype=np.float32)
# concat_vector = np.hstack((contig_query_vector, contig_img_vector))
# np.savez('res/data/CONCAT_가공식품', concat_vector)


# index path
title_index_path = 'res/model/index/title.h5'
img_index_path = 'res/model/index/image.h5'
concat_index_path = 'res/model/index/concat.h5'

# index_handler
index_handler = IndexHandler()

# make & save index
index_handler.make_index(arr_title_vector)
index_handler.save_index(title_index_path)

index_handler.make_index(arr_img_vector)
index_handler.save_index(img_index_path)

index_handler.make_index(arr_concat_vector)
index_handler.save_index(concat_index_path)

True
True
True


In [7]:
from index_handle import IndexHandler
index_handler = IndexHandler()

# load index
title_index = index_handler.load_index(title_index_path)
img_index = index_handler.load_index(img_index_path)
concat_index = index_handler.load_index(concat_index_path)

## 5-1. Intersection filter (img ∩ text)

In [8]:
from filter_handle import FilterHandler
filter_handler = FilterHandler(cleaned_arr_title, arr_weight, title_index, arr_title_vector, img_index, arr_img_vector, concat_index, arr_concat_vector)

In [9]:
# 미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g
query_title = cleaned_arr_title[15000]

prod_filter = filter_handler.get_filter('intersection')
sim_titles = prod_filter.do(query_title)

print('제목 : {0}'.format(query_title[0]))
print('-'*50)
print('유사 상품 제목')
print(sim_titles.flatten())

제목 : 미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임
--------------------------------------------------
유사 상품 제목
['미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임' '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임'
 '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임' '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임'
 '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임' '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임'
 '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임' '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임 통'
 '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임 통' '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임 통'
 '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임 통' '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임 통'
 '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임 통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통'
 '원더랜드 미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통' '원더랜드 미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 통'
 '미소반 환상궁합 고추냉이잎 절

## 5-2. Sequential filter(img->text)

In [10]:
prod_filter = filter_handler.get_filter('sequential')
sim_titles = prod_filter.do(query_title, filter_type='img')

print('제목 : {0}'.format(query_title[0]))
print('-'*50)
print('유사 상품 제목')
print(sim_titles.flatten())

제목 : 미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임
--------------------------------------------------
유사 상품 제목
['미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임' '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임' '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임' '원더랜드 미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임' '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임'
 '고추냉이 냉이잎 냉이 미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓'
 '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임'
 '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임'
 '원더랜드 미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임'
 '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '상하식품 환상궁합 장아찌 절임 돋우는 국산 미소'
 '국산 절임 신선식품 방풍나물절임 돋우는 미소반 상하식품'
 '절임 방풍나물절임 미소반 상하식품 신선식품 반찬 장아찌 국산 돌산갓 깔끔향긋 돋우는'
 '국산 장아찌 상하식품 방풍나물절임 반찬 절임 미소반 돌산갓 돋우는 깔끔향긋 신선식품'
 '환상궁합 국산 고추냉이잎 입맛 신선식품 돌산갓 돋우는 반찬 미소반 장아찌 상하식품'
 '국산 돌산갓 미소반 깔끔향긋 입맛 상하식품 돋우는 방풍나' '반찬 자연산 절임 돌산갓 절임 신선식품 상하식품 국산 장'
 '절임 신선식품 미소반 장아찌 국산 깔끔향긋 명이나물 울릉' '미소반 돋우는 신선식품 고추냉이잎 국산 환상궁합 절임 

## 5-3. Sequential filter(text->img)

In [11]:
prod_filter = filter_handler.get_filter('sequential')
sim_titles = prod_filter.do(query_title, filter_type='title')

print('제목 : {0}'.format(query_title[0]))
print('-'*50)
print('유사 상품 제목')
print(sim_titles.flatten())

제목 : 미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임
--------------------------------------------------
유사 상품 제목
['미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임' '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임'
 '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임' '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임'
 '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임' '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임'
 '미소반 엄선한 재료와 소스의 환상궁합 고추냉이잎 절임' '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임' '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임' '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임'
 '원더랜드 미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임' '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓 절임'
 '고추냉이 냉이잎 냉이 미소반 환상궁합 고추냉이잎 절임 입맛 돋우는 돌산갓'
 '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임'
 '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임'
 '원더랜드 미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임'
 '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '미소반 환상궁합 고추냉이잎 절임 깔끔향긋 방풍나물절임'
 '미소반 환상궁합 고추냉이잎 절임 깔끔향긋 방풍나물절임' '미소반 환상궁합 고추냉이잎 절임 깔끔향긋 방풍나물절임'
 '미소반 환상궁합 고추냉이잎 절임 깔끔

## 5-4. Concat filter

In [12]:
prod_filter = filter_handler.get_filter('concat')
sim_titles = prod_filter.do(query_title)

print('제목 : {0}'.format(query_title[0]))
print('-'*50)
print('유사 상품 제목')
print(sim_titles.flatten())

제목 : 미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임
--------------------------------------------------
유사 상품 제목
['미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임'
 '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임'
 '원더랜드 미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임'
 '미소반 입맛 돋우는 돌산갓 절임 깔끔향긋 방풍나물절임' '국산 절임 신선식품 방풍나물절임 돋우는 미소반 상하식품'
 '절임 방풍나물절임 미소반 상하식품 신선식품 반찬 장아찌 국산 돌산갓 깔끔향긋 돋우는'
 '국산 장아찌 상하식품 방풍나물절임 반찬 절임 미소반 돌산갓 돋우는 깔끔향긋 신선식품'
 '국산 돌산갓 미소반 깔끔향긋 입맛 상하식품 돋우는 방풍나']
